# 如何组合提示词
:::info 前提条件
本指南假定您已熟悉以下概念：- [提示模板](/docs/concepts/prompt_templates)
:::
LangChain 为用户提供了友好的接口，用于将[提示模板](/docs/concepts/prompt_templates/)的不同部分组合在一起。您可以使用字符串提示或聊天提示来实现这一点。通过这种方式构建提示，可以轻松实现组件的重复使用。

## 字符串提示组合
在处理字符串提示时，每个模板会被连接在一起。您可以直接使用提示或字符串（列表中的第一个元素必须是一个提示）。

In [1]:
from langchain_core.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny"
    + "\n\nand in {language}"
)

prompt

PromptTemplate(input_variables=['language', 'topic'], template='Tell me a joke about {topic}, make it funny\n\nand in {language}')

In [2]:
prompt.format(topic="sports", language="spanish")

'Tell me a joke about sports, make it funny\n\nand in spanish'

## 聊天提示词构成

聊天提示由一系列消息组成。与上述示例类似，我们可以将聊天提示模板进行串联。每个新元素都会成为最终提示中的一条新消息。
首先，让我们使用 [`SystemMessage`](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.system.SystemMessage.html) 初始化一个 [`ChatPromptTemplate`](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)。

In [3]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

prompt = SystemMessage(content="You are a nice pirate")

然后，你可以轻松地将其与其他消息*或*消息模板组合，创建一个流程。当没有需要格式化的变量时，使用 `Message`；当存在需要格式化的变量时，使用 `MessageTemplate`。你也可以直接使用字符串（注意：这会自动推断为 [`HumanMessagePromptTemplate`](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.HumanMessagePromptTemplate.html)。）

In [4]:
new_prompt = (
    prompt + HumanMessage(content="hi") + AIMessage(content="what?") + "{input}"
)

在底层，这会创建一个 ChatPromptTemplate 类的实例，因此你可以像之前一样使用它！

In [5]:
new_prompt.format_messages(input="i said hi")

[SystemMessage(content='You are a nice pirate'),
 HumanMessage(content='hi'),
 AIMessage(content='what?'),
 HumanMessage(content='i said hi')]

## 使用PipelinePrompt

LangChain 包含一个名为 [`PipelinePromptTemplate`](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html) 的类，当您想要复用提示的某些部分时，它会非常有用。PipelinePrompt 主要由两部分组成：
- 最终提示：返回的最终提示- 流水线提示：一个由元组组成的列表，每个元组包含一个字符串名称和一个提示模板。每个提示模板将被格式化后，作为同名变量传递给后续的提示模板。

In [6]:
from langchain_core.prompts import PipelinePromptTemplate, PromptTemplate

full_template = """{introduction}

{example}

{start}"""
full_prompt = PromptTemplate.from_template(full_template)

introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)

pipeline_prompt.input_variables

['person', 'example_a', 'example_q', 'input']

In [7]:
print(
    pipeline_prompt.format(
        person="Elon Musk",
        example_q="What's your favorite car?",
        example_a="Tesla",
        input="What's your favorite social media site?",
    )
)

You are impersonating Elon Musk.

Here's an example of an interaction:

Q: What's your favorite car?
A: Tesla

Now, do this for real!

Q: What's your favorite social media site?
A:


## 后续步骤
你现在已经学会了如何组合提示词。
接下来，请查阅本节中关于提示模板的其他操作指南，例如[如何向提示模板添加少量示例](/docs/how_to/few_shot_examples_chat)。